In [ ]:
!pip install scipy==1.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 1.9 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.2.1 which is incompatible.
jaxlib 0.3.14+cuda11.cudnn805 requires scipy>=1.5, but you have scipy 1.2.1 which is incompatible.
jax 0.3.14 requires scipy>=1.5, but you have scipy 1.2.1 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/My Drive/CS523/Project/face_classification/src')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/CS523/Project/face_classification/src

In [ ]:
#from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import *
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from models.cnn import mini_XCEPTION
from utils.datasets import DataManager
from utils.datasets import split_data
from utils.preprocessor import preprocess_input

import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
print(tf.__version__)

2.8.2


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# parameters
batch_size = 32
num_epochs = 10000
input_shape = (64, 64, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
base_path = '../trained_models/emotion_models/'

In [ ]:
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [ ]:
# model parameters/compilation
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 62, 62, 8)    72          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 62, 62, 8)   32          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 62, 62, 8)    0           ['batch_normalization[0][0]']

In [ ]:
datasets = ['fer2013']
for dataset_name in datasets:
    print('Training dataset:', dataset_name)

    # callbacks
    log_file_path = base_path + dataset_name + '_emotion_training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
    trained_models_path = base_path + dataset_name + '_mini_XCEPTION'
    model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

    # loading dataset
    data_loader = DataManager(dataset_name, image_size=input_shape[:2])
    faces, emotions = data_loader.get_data()
    faces = preprocess_input(faces)
    num_samples, num_classes = emotions.shape
    train_data, val_data = split_data(faces, emotions, validation_split)
    train_faces, train_emotions = train_data
    # model.fit_generator(data_generator.flow(train_faces, train_emotions,
    #                                         batch_size),
    #                     steps_per_epoch=len(train_faces) / batch_size,
    #                     epochs=num_epochs, verbose=1, callbacks=callbacks,
    #                     validation_data=val_data)
    history = model.fit(data_generator.flow(train_faces, train_emotions,
                                            batch_size),
                        steps_per_epoch=len(train_faces) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=val_data)

Training dataset: fer2013
Epoch 1/10000
898/897 [==============================] - ETA: 0s - loss: 1.2092 - accuracy: 0.5467
Epoch 1: val_loss improved from inf to 1.17472, saving model to ../trained_models/increased_modules_experiments_5/fer2013_mini_XCEPTION.01-0.55.hdf5
897/897 [==============================] - 29s 33ms/step - loss: 1.2092 - accuracy: 0.5467 - val_loss: 1.1747 - val_accuracy: 0.5545 - lr: 0.0010
Epoch 2/10000
898/897 [==============================] - ETA: 0s - loss: 1.1864 - accuracy: 0.5525
Epoch 2: val_loss improved from 1.17472 to 1.16601, saving model to ../trained_models/increased_modules_experiments_5/fer2013_mini_XCEPTION.02-0.56.hdf5
897/897 [==============================] - 31s 34ms/step - loss: 1.1864 - accuracy: 0.5525 - val_loss: 1.1660 - val_accuracy: 0.5577 - lr: 0.0010
Epoch 3/10000
897/897 [============================>.] - ETA: 0s - loss: 1.1589 - accuracy: 0.5636
Epoch 3: val_loss did not improve from 1.16601
897/897 [===========================